# Import Library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import time
import warnings
from tqdm.auto import tqdm

# Konfigurasi tampilan dan warning
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# Configuration & Constants

In [2]:
# Thresholds
QUALITY_THRESHOLD = 50
VALUATION_THRESHOLD = 50

# Scoring Weights
WEIGHTS = {
    'profitability': 0.40,
    'financial_health': 0.25,
    'growth': 0.20,
    'efficiency': 0.15
}

# System Config
REQUEST_DELAY = 0.1  # Detik (untuk menghindari rate limiting yfinance)
SP500_SOURCE_URL = 'https://raw.githubusercontent.com/datasets/s-and-p-500-companies/refs/heads/main/data/constituents.csv'

# Helper Functions - Financial Calculations

In [3]:
# Return on Equity (%)
def calculate_roe(net_income, shareholder_equity):
    if shareholder_equity and shareholder_equity > 0:
        return (net_income / shareholder_equity) * 100
    return 0

# Return on Assets (%)
def calculate_roa(net_income, total_assets):
    if total_assets and total_assets > 0:
        return (net_income / total_assets) * 100
    return 0

# Debt to Equity Ratio
def calculate_debt_to_equity(total_debt, shareholder_equity):
    if shareholder_equity and shareholder_equity > 0:
        return total_debt / shareholder_equity
    return float('inf')

# Current Ratio
def calculate_current_ratio(current_assets, current_liabilities):
    if current_liabilities and current_liabilities > 0:
        return current_assets / current_liabilities
    return 0

# Net Profit Margin (%)
def calculate_profit_margin(net_income, revenue):
    if revenue and revenue > 0:
        return (net_income / revenue) * 100
    return 0

# Operating Margin (%)
def calculate_operating_margin(operating_income, revenue):
    if revenue and revenue > 0:
        return (operating_income / revenue) * 100
    return 0

# PEG Ratio
def calculate_peg_ratio(pe_ratio, eps_growth):
    if eps_growth and eps_growth > 0:
        return pe_ratio / eps_growth
    return float('inf')

# Free Cash Flow Yield (%)
def calculate_fcf_yield(free_cash_flow, market_cap):
    if market_cap and market_cap > 0:
        return (free_cash_flow / market_cap) * 100
    return 0

# Menghitung pertumbuhan YoY dari data laporan keuangan
def calculate_growth(financials, metric_name):
    try:
        if metric_name in financials.index:
            metric_data = financials.loc[metric_name]
            if len(metric_data) >= 2:
                current = metric_data.iloc[0]
                previous = metric_data.iloc[1]
                if previous and previous > 0:
                    return ((current - previous) / previous) * 100
        return 0
    except:
        return 0

# Helper Functions - Data Acquisition

In [4]:
# Mengambil daftar ticker S&P 500 terbaru dari GitHub
def get_sp500_tickers():
    try:
        print(f"Mengunduh daftar S&P 500 dari: {SP500_SOURCE_URL}")
        df_sp500 = pd.read_csv(SP500_SOURCE_URL)
        tickers = df_sp500['Symbol'].str.replace('.', '-', regex=False).tolist()
        print(f"Berhasil mengambil {len(tickers)} ticker S&P500")
        return tickers
    except Exception as e:
        print(f"GAGAL mengunduh daftar S&P 500: {e}")
        raise Exception("Tidak dapat melanjutkan tanpa daftar ticker")

# Mengambil data fundamental lengkap untuk satu ticker menggunakan yfinance
# Mengembalikan dictionary data atau None jika gagal
def get_fundamental_data(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        financials = stock.financials
        balance_sheet = stock.balance_sheet
        cash_flow = stock.cashflow

        # Skip jika data utama tidak lengkap
        if financials.empty or balance_sheet.empty:
            return None

        # Extract Income Statement
        if 'Total Revenue' in financials.index:
            revenue = financials.loc['Total Revenue'].iloc[0]
        elif 'Revenue' in financials.index:
            revenue = financials.loc['Revenue'].iloc[0]
        else:
            return None

        if 'Net Income' in financials.index:
            net_income = financials.loc['Net Income'].iloc[0]
        else:
            return None

        # Operating income fallback
        if 'Operating Income' in financials.index:
            operating_income = financials.loc['Operating Income'].iloc[0]
        else:
            operating_income = net_income * 0.8

        # Extract Balance Sheet
        if 'Total Assets' in balance_sheet.index:
            total_assets = balance_sheet.loc['Total Assets'].iloc[0]
        else:
            return None

        # Fallbacks for assets/liabilities/equity
        current_assets = balance_sheet.loc['Current Assets'].iloc[0] if 'Current Assets' in balance_sheet.index else total_assets * 0.3
        current_liabilities = balance_sheet.loc['Current Liabilities'].iloc[0] if 'Current Liabilities' in balance_sheet.index else total_assets * 0.2

        if 'Stockholders Equity' in balance_sheet.index:
            shareholder_equity = balance_sheet.loc['Stockholders Equity'].iloc[0]
        elif 'Total Equity' in balance_sheet.index:
            shareholder_equity = balance_sheet.loc['Total Equity'].iloc[0]
        else:
            shareholder_equity = total_assets * 0.5

        if 'Total Debt' in balance_sheet.index:
            total_debt = balance_sheet.loc['Total Debt'].iloc[0]
        elif 'Long Term Debt' in balance_sheet.index:
            total_debt = balance_sheet.loc['Long Term Debt'].iloc[0]
        else:
            total_debt = 0

        # Extract Cash Flow
        if not cash_flow.empty:
            if 'Operating Cash Flow' in cash_flow.index:
                operating_cash_flow = cash_flow.loc['Operating Cash Flow'].iloc[0]
            elif 'Cash Flow From Continuing Operating Activities' in cash_flow.index:
                operating_cash_flow = cash_flow.loc['Cash Flow From Continuing Operating Activities'].iloc[0]
            else:
                operating_cash_flow = net_income

            if 'Capital Expenditure' in cash_flow.index:
                capital_expenditure = abs(cash_flow.loc['Capital Expenditure'].iloc[0])
            else:
                capital_expenditure = 0

            free_cash_flow = operating_cash_flow - capital_expenditure
        else:
            free_cash_flow = net_income

        # Market Data & Growth
        market_cap = info.get('marketCap', 0)
        pe_ratio = info.get('trailingPE', 0) or info.get('forwardPE', 0)
        pb_ratio = info.get('priceToBook', 0)
        ps_ratio = info.get('priceToSalesTrailing12Months', 0)
        ev_ebitda = info.get('enterpriseToEbitda', 0)

        revenue_growth = calculate_growth(financials, 'Total Revenue') or calculate_growth(financials, 'Revenue')
        eps_growth = info.get('earningsGrowth', 0) * 100 if info.get('earningsGrowth') else 0

        # Calculate Ratios
        data = {
            'ticker': ticker,
            'company_name': info.get('longName', 'N/A'),
            'sector': info.get('sector', 'N/A'),
            'industry': info.get('industry', 'N/A'),

            # Metrics
            'roe': calculate_roe(net_income, shareholder_equity),
            'roa': calculate_roa(net_income, total_assets),
            'profit_margin': calculate_profit_margin(net_income, revenue),
            'operating_margin': calculate_operating_margin(operating_income, revenue),
            'debt_to_equity': calculate_debt_to_equity(total_debt, shareholder_equity),
            'current_ratio': calculate_current_ratio(current_assets, current_liabilities),
            'revenue_growth': revenue_growth,
            'eps_growth': eps_growth,
            'asset_turnover': (revenue / total_assets) * 100 if total_assets > 0 else 0,

            # Valuation
            'pe_ratio': pe_ratio if pe_ratio else 0,
            'pb_ratio': pb_ratio if pb_ratio else 0,
            'ps_ratio': ps_ratio if ps_ratio else 0,
            'peg_ratio': calculate_peg_ratio(pe_ratio, eps_growth),
            'ev_ebitda': ev_ebitda if ev_ebitda else 0,
            'fcf_yield': calculate_fcf_yield(free_cash_flow, market_cap),

            # Raw Data (Reference)
            'market_cap': market_cap,
            'revenue': revenue,
            'net_income': net_income,
            'free_cash_flow': free_cash_flow
        }
        return data

    except (KeyError, IndexError, Exception):
        return None

# Helper Functions - Scoring & Labeling

In [5]:
# Helper function untuk memberikan skor diskrit berdasarkan threshold
def score_metric(value, thresholds, scores, reverse=False):
    if pd.isna(value) or value == float('inf') or value == 0:
        return 0

    if reverse: # Lower is better (e.g. Debt, PER)
        if value <= thresholds[2]: return scores[2]
        elif value <= thresholds[1]: return scores[1]
        elif value <= thresholds[0]: return scores[0]
        return 0
    else: # Higher is better (e.g. ROE, Growth)
        if value >= thresholds[2]: return scores[2]
        elif value >= thresholds[1]: return scores[1]
        elif value >= thresholds[0]: return scores[0]
        return 0

# Menghitung skor kualitas fundamental (0-100)
def calculate_quality_score(stock_data):
    score = 0
    w = WEIGHTS

    # 1. PROFITABILITY (40%)
    p_score = 0
    p_score += score_metric(stock_data['roe'], [8, 12, 16], [0, 20, 40])
    p_score += score_metric(stock_data['roa'], [4, 6, 10], [0, 15, 30])
    p_score += score_metric(stock_data['profit_margin'], [4, 8, 12], [0, 15, 30])
    p_score += score_metric(stock_data['operating_margin'], [6, 10, 15], [0, 15, 30])
    score += (p_score / 130) * 100 * w['profitability']

    # 2. FINANCIAL HEALTH (25%)
    h_score = 0
    h_score += score_metric(stock_data['debt_to_equity'], [2.0, 1.0, 0.5], [0, 20, 40], reverse=True)
    h_score += score_metric(stock_data['current_ratio'], [0.8, 1.2, 1.8], [0, 20, 40])
    score += (h_score / 80) * 100 * w['financial_health']

    # 3. GROWTH (20%)
    g_score = 0
    g_score += score_metric(stock_data['revenue_growth'], [3, 8, 15], [0, 20, 40])
    g_score += score_metric(stock_data['eps_growth'], [3, 8, 15], [0, 20, 40])
    score += (g_score / 80) * 100 * w['growth']

    # 4. EFFICIENCY (15%)
    e_score = score_metric(stock_data['asset_turnover'], [15, 30, 50], [0, 50, 100])
    score += e_score * w['efficiency']

    return min(100, max(0, score))

# Menghitung skor valuasi (0-100)
# Higher Score = More Undervalued
def calculate_valuation_score(stock_data):
    score = 0

    # P/E Ratio (20%)
    pe = stock_data['pe_ratio']
    if 0 < pe < 100:
        if pe < 12: score += 25
        elif pe < 18: score += 20
        elif pe < 25: score += 15
        elif pe < 35: score += 10
        else: score += 5

    # P/B Ratio (20%)
    pb = stock_data['pb_ratio']
    if pb > 0:
        if pb < 1.2: score += 25
        elif pb < 2.0: score += 20
        elif pb < 3.0: score += 15
        elif pb < 5.0: score += 10
        else: score += 5

    # PEG Ratio (15%)
    peg = stock_data['peg_ratio']
    if 0 < peg < 10:
        if peg < 0.7: score += 20
        elif peg < 1.2: score += 15
        elif peg < 2.0: score += 10
        else: score += 5

    # FCF Yield (20%)
    score += score_metric(stock_data['fcf_yield'], [2, 4, 6], [0, 10, 20])

    # EV/EBITDA (15%)
    ev = stock_data['ev_ebitda']
    if 0 < ev < 30:
        if ev < 8: score += 15
        elif ev < 12: score += 12
        elif ev < 16: score += 8
        else: score += 4

    # P/S Ratio (10%)
    ps = stock_data['ps_ratio']
    if 0 < ps < 20:
        if ps < 1: score += 10
        elif ps < 2: score += 8
        elif ps < 4: score += 5
        else: score += 2

    return min(100, max(0, score))

# Menentukan kuadran saham
def assign_label(quality_score, valuation_score):
    q_label = "good_quality" if quality_score >= QUALITY_THRESHOLD else "bad_quality"
    v_label = "undervalued" if valuation_score >= VALUATION_THRESHOLD else "overvalued"
    return f"{q_label}_{v_label}"

# Validation Functions

In [6]:
# Membersihkan infinite values menjadi NaN
def clean_dataset(df):
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    for col in numeric_columns:
        df[col] = df[col].replace([np.inf, -np.inf], np.nan)
    return df

# Melakukan pengecekan statistik dataset
def validate_dataset(df):
    if df.empty:
        print("Dataset kosong, tidak dapat divalidasi")
        return df

    print(f"Shape: {df.shape}")

    print(f"\nFinal Columns:")
    for i, col in enumerate(df.columns, 1):
        print(f"{i:2d}. {col}")

    print("\nStatistik Missing Values")
    missing_data = df.isnull().sum()
    missing_cols = missing_data[missing_data > 0]

    if len(missing_cols) > 0:
        print(f"{'Kolom':<20} {'Jumlah':<10} {'Persen':<10}")
        for col, count in missing_cols.items():
            pct = (count / len(df)) * 100
            print(f"{col:<20} {count:<10} {pct:.1f}%")
    else:
        print("Tidak ada missing values")

    print("\nDistribusi Label")
    label_counts = df['label'].value_counts()
    for label, count in label_counts.items():
        pct = (count / len(df)) * 100
        print(f"{label}: {count} saham ({pct:.1f}%)")

    print(f"\nStatistik Deskriptif:")
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if not numeric_cols.empty:
        print(df[numeric_cols].describe())

    return df

# MAIN EXECUTION PIPELINE

In [7]:
# Mendapatkan ticker
tickers = get_sp500_tickers()

# Mengumpulkan data
all_stock_data = []
successful_count = 0

print("\nData Collection Process")
for ticker in tqdm(tickers, desc="Processing Stocks", unit="stock"):
    stock_data = get_fundamental_data(ticker)

    if stock_data:
        # Calculate Scores
        q_score = calculate_quality_score(stock_data)
        v_score = calculate_valuation_score(stock_data)

        # Assign Label
        label = assign_label(q_score, v_score)

        # Update Record
        stock_data.update({
            'quality_score': q_score,
            'valuation_score': v_score,
            'label': label
        })

        all_stock_data.append(stock_data)
        successful_count += 1

    time.sleep(REQUEST_DELAY)

if not all_stock_data:
    raise Exception("Tidak ada data yang berhasil dikumpulkan.")

df = pd.DataFrame(all_stock_data)
print(f"\nBerhasil dikumpulkan: {successful_count}")
print(f"Success Rate: {(successful_count/len(tickers))*100:.1f}%")

Mengunduh daftar S&P 500 dari: https://raw.githubusercontent.com/datasets/s-and-p-500-companies/refs/heads/main/data/constituents.csv
Berhasil mengambil 503 ticker S&P500

Data Collection Process


ERROR:yfinance:HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: WBA"}}}



Berhasil dikumpulkan: 502
Success Rate: 99.8%


In [8]:
# Menghapus kolom yang tidak diperlukan
cols_to_drop = ['company_name', 'industry', 'quality_score', 'valuation_score']
df_cleaned = df.drop(columns=cols_to_drop, errors='ignore')
df_final = clean_dataset(df_cleaned)

# Validate
final_dataset = validate_dataset(df_final)

Shape: (502, 22)

Final Columns:
 1. ticker
 2. sector
 3. roe
 4. roa
 5. profit_margin
 6. operating_margin
 7. debt_to_equity
 8. current_ratio
 9. revenue_growth
10. eps_growth
11. asset_turnover
12. pe_ratio
13. pb_ratio
14. ps_ratio
15. peg_ratio
16. ev_ebitda
17. fcf_yield
18. market_cap
19. revenue
20. net_income
21. free_cash_flow
22. label

Statistik Missing Values
Kolom                Jumlah     Persen    
roe                  6          1.2%
roa                  6          1.2%
debt_to_equity       35         7.0%
revenue_growth       6          1.2%
asset_turnover       6          1.2%
peg_ratio            202        40.2%
fcf_yield            4          0.8%
revenue              6          1.2%
net_income           6          1.2%
free_cash_flow       4          0.8%

Distribusi Label
bad_quality_undervalued: 164 saham (32.7%)
good_quality_overvalued: 138 saham (27.5%)
bad_quality_overvalued: 117 saham (23.3%)
good_quality_undervalued: 83 saham (16.5%)

Statistik Deskript

# Export Data

In [9]:
# Export ke CSV
file_name = "sp500_quadrant_dataset.csv"
final_dataset.to_csv(file_name, index=False)

print(f"Dataset berhasil disimpan sebagai: {file_name}")

Dataset berhasil disimpan sebagai: sp500_quadrant_dataset.csv
